# Установка нужных библиотек

# Импорт библиотек

In [1]:
import os
import json
import glob
import torch

import matplotlib.pyplot as plt

from src.mylib.models.models import BaselineModel, MHAModel
from src.mylib.train import Trainer

file = os.path.abspath('')

2024-04-03 15:20:26.102276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 15:20:26.883476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Работа с данными

Начальные параметры

In [2]:
batch_size = 64

# Длина окна
window_length_seconds = 5 
sample_rate = 64
window_length = window_length_seconds * sample_rate

# Расстояние между двумя окнами
hop_length_seconds = 1
hop_length = sample_rate * hop_length_seconds

# Количество ложные стимулов
number_of_mismatch = 4

In [3]:
experiment_folder = os.path.dirname(file)

# Load the config file
with open(os.path.join(experiment_folder, "src/mylib/utils/config.json")) as file_path:
    config = json.load(file_path)

# Path to the dataset, which is already split to train, val, test
data_folder = os.path.join(config["dataset_folder"], config['derivatives_folder'], config["split_folder"])

# Пути к данным тренировочным, валидационным и тестовым данным
train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]
val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]
test_files = [x for x in glob.glob(os.path.join(data_folder, "test_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]

## Обучение 

In [4]:
# model = BaselineModel()
args = {"window_length" : window_length, "hop_length" : hop_length, "number_of_mismatch" : number_of_mismatch, "batch_size" : batch_size, 
        "max_files" : None}
for model in [BaselineModel, MHAModel]:
    model = model()
    print(f"Model: {model.__class__.__name__}")
    
    trainer = Trainer(
        model, train_files, val_files, test_files, args, torch.optim.Adam(model.parameters(), lr=0.001) 
    )

    trainer.train_model(epochs=2, run_name=f"{model.__class__.__name__}", eps=1e-5)
    trainer.test(window_length, hop_length, number_of_mismatch, None)
    print("-----" * 10)

Model: BaselineModel
EPOCH 1:
  batch 100 loss: 0.10230839826722066
  batch 200 loss: 0.00040566950565384176
  batch 300 loss: 0.00019768533426147594
  batch 400 loss: 2.5900446552251656e-06
LOSS train 2.5900446552251656e-06 valid 4.137537594234935e-05
EPOCH 2:
  batch 100 loss: 0.0001946311015124791
  batch 200 loss: 7.989311695837387e-06
LOSS train 7.989311695837387e-06 valid 3.112558865379723e-05
sub-036
    Mean accuracy per subject: 100.0
sub-076
    Mean accuracy per subject: 100.0
sub-017
    Mean accuracy per subject: 100.0
sub-064
    Mean accuracy per subject: 100.0
sub-042
    Mean accuracy per subject: 100.0
sub-029
    Mean accuracy per subject: 100.0
sub-078
    Mean accuracy per subject: 100.0
sub-022
    Mean accuracy per subject: 100.0
sub-005
    Mean accuracy per subject: 100.0
sub-011
    Mean accuracy per subject: 99.87893462469734
sub-014
    Mean accuracy per subject: 100.0
sub-013
    Mean accuracy per subject: 99.87893462469734
sub-033
    Mean accuracy per sub